### Import Libraries

In [1512]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
import dash_table
import folium
import requests

### Define Styles

In [1513]:
##----------------------------------------STYLES----------------------------------------##
style_card = {"width": "18rem", 'margin': '20px', 'display': 'inline-block', 'vertical-align': 'top', 'text-align': 'center', 'padding': '10px', 'background-color': '#ffffff', 'color': 'black', 'font-family': 'Arial, sans-serif'}
style_main = {'max-width':'1400px', 'text-align': 'center', 'padding':'30px', 'margin': '0 auto'}
style_title = {'max-width':'1400px','margin': '0 auto', 'font-family': 'Arial, sans-serif', 'background-color': '#223577', 'padding': '30px'}
home_button_style = {'margin': '20px', 'background-color': '#223577', 'color': 'white', 'font-family': 'Arial, sans-serif', 'cursor':'pointer', 'font-size':'16px', 'border-radius':'7px'}
style_IPL = {'color': 'white', 'font-family': 'Arial, sans-serif'}
style_text = {'color': 'black', 'font-family': 'Arial, sans-serif', 'padding':'10px', 'margin':'10px'}

### Define Home Screen

In [1514]:
external_stylesheets = ['assets/style.css', dbc.themes.BOOTSTRAP]
app = dash.Dash(__name__, external_stylesheets = external_stylesheets)


# define a card component
card1 = dbc.Card(
    [
        dbc.CardImg(src="./assets/batting.png", top=True, style={'height':'16rem'}),
        dbc.CardBody(
            [
                html.H4("Batsmen", className="card-title"),
                html.P(
                    "Batsmen summary of all the seasons of IPL.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/batting"),
            ]
        ),
    ],
    style=style_card,
)

card2 = dbc.Card(
    [
        dbc.CardImg(src="./assets/bowling.png", top=True, style={'height':'16rem'}),
        dbc.CardBody(
            [
                html.H4("Bowler", className="card-title"),
                html.P(
                    "Bowlers summary of all the seasons of IPL.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/bowling"),
            ]
        ),
    ],
    style=style_card,
)

card3 = dbc.Card(
    [
        dbc.CardImg(src="./assets/stadium.jpg", top=True, style={'height':'16rem'}),
        dbc.CardBody(
            [
                html.H4("Stadiums", className="card-title"),
                html.P(
                    "Stadium statistics of all the seasons of IPL.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/stadium"),
            ]
        ),
    ],
    style=style_card,
)

card4 = dbc.Card(
    [
        dbc.CardImg(src="./assets/ranking.png", top=True, style={'height':'17.5rem'}),
        dbc.CardBody(
            [
                html.H4("Ranking", className="card-title"),
                html.P(
                    "Ranking of all the teams of IPL.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/rank"),
            ]
        ),
    ],
    style=style_card,
)

card5 = dbc.Card(
    [
        dbc.CardImg(src="./assets/team.png", top=True, style={'height':'16rem'}),
        dbc.CardBody(
            [
                html.H4("Teams", className="card-title"),
                html.P(
                    "Team statistics of all the seasons of IPL.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/team"),
            ]
        ),
    ],
    style=style_card,
)

card6 = dbc.Card(
    [
        dbc.CardImg(src="./assets/summary.jpg", top=True, style={'height':'16rem'}),
        dbc.CardBody(
            [
                html.H4("Match Summary", className="card-title"),
                html.P(
                    "Match Summary of all the seasons of IPL.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/summary"),
            ]
        ),
    ],
    style=style_card,
)


# Define the layout for the home page
home_layout = html.Div(children=[
    html.Div(style=style_title, children=[
    html.H1('Indian Premier League', style={'color': 'white', 'font-family': 'Arial, sans-serif'}),
]),
    card1, card2, card3, card4, card5, card6
    
], style=style_main)



### Batsmen Layout

In [1515]:
run_per_batsman_per_season = pd.read_csv('./Data/Total_run_per_batsman_per_Season.csv')
batter = sorted(run_per_batsman_per_season['batter'].unique())
top_batsmen_per_season = run_per_batsman_per_season.groupby(['Season']).max().reset_index()

batsmen_layout  = html.Div(children=[
    html.Div(style=style_title, children=[
    html.H1('Indian Premier League', style=style_IPL),]),
    html.A(html.Button('Go back to home', style=home_button_style), href='/'),
    html.Div([
        html.H1("Total Runs per Season by player", style=style_text),
        dcc.Dropdown(
            id='player-dropdown',
            options=[{'label': player, 'value': player} for player in batter],
            value=batter[0]
        ),
        dcc.Graph(id='line-chart-batsman')
        ]),
], style=style_main)

@app.callback(
    Output('line-chart-batsman', 'figure'),
    [Input('player-dropdown', 'value')]
)
def update_line_chart1(selected_player):
    # Filter the dataframe based on the selected team
    filtered_df = run_per_batsman_per_season[run_per_batsman_per_season['batter'] == selected_player]
    filtered_df.sort_values('Season', inplace=True)
    # Create a line chart using Plotly Express
    fig = px.line(filtered_df, x='Season', y='batsman_run', title=f'Total runs per season for {selected_player}')
    return fig

### Bowler Layout

In [1516]:
num_wicket_per_bowler = pd.read_csv('./Data/num_wickets_per_bowler.csv')
bowler = sorted(num_wicket_per_bowler['bowler'].unique())


bowler_layout  = html.Div(children=[
    html.Div(style=style_title,children=[
    html.H1('Indian Premier League', style=style_IPL),]),
    html.A(html.Button('Go back to home', style=home_button_style), href='/'),
    html.Div([
        html.H1("Total Wickets Taken by Player per Season", style = style_text),
        dcc.Dropdown(
            id='player-dropdown-bowler',
            options=[{'label': player, 'value': player} for player in bowler],
            value=bowler[0]
        ),
        dcc.Graph(id='line-chart-bowler')
        ]),
], style=style_main)

@app.callback(
    Output('line-chart-bowler', 'figure'),
    [Input('player-dropdown-bowler', 'value')]
)
def update_line_chart2(selected_player):
    # Filter the dataframe based on the selected team
    filtered_df = num_wicket_per_bowler[num_wicket_per_bowler['bowler'] == selected_player]
    filtered_df.sort_values('Season', inplace=True)
    # Create a line chart using Plotly Express
    fig = px.line(filtered_df, x='Season', y='Wickets', title=f'Total wickets per season for {selected_player}')
    return fig


### Stadium Layout

In [1517]:

matches_per_city = pd.read_csv('./Data//num_matches_per_city.csv')
# find the list of unique cities

season_options = [{'label': str(season), 'value': season} for season in matches_per_city['Season'].unique()]

#define layout
stadium_layout = html.Div(children=[
        html.Div(style=style_title, children=[
        html.H1('Indian Premier League', style=style_IPL),]),
        html.A(html.Button('Go back to home', style=home_button_style), href='/'),
        html.Div(children=[
        html.H1(children='Number of Matches Played in Each City', style=style_text),
        dcc.Dropdown(
            id='season-dropdown-2',
            options=season_options,
            value=matches_per_city['Season'].unique()[0]
        ),
        dcc.Graph(
            id='matches-per-city',
            figure={}
        ),
        html.H2(children='IPL Stadiums Stats on Map', style=style_text),
        html.Iframe(id='india-map', width='100%', height='600')
    ])
], style=style_main)




# Define the callback function
@app.callback(
    dash.dependencies.Output('matches-per-city', 'figure'),
    dash.dependencies.Output('india-map', 'srcDoc'),
    [dash.dependencies.Input('season-dropdown-2', 'value')]
)
def update_figure(selected_season):
    # Filter the data based on the selected season
    filtered_data = matches_per_city[matches_per_city['Season'] == selected_season].sort_values('Count')
    india_map = folium.Map(location=[20.5937, 78.9629], no_touch=True, tiles='stamentoner', min_zoom=5, max_zoom=5)
    india_map.fit_bounds([[6.462, 68.1097], [35.494, 97.395]])
    api_key = "25f3711d51ba44651a0369311124f949"
    for city in filtered_data['City']:
        geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={api_key}"
        geo_response = requests.get(geo_url)
        geo_data = geo_response.json()
        lat = geo_data[0]['lat']
        lon = geo_data[0]['lon']
        folium.CircleMarker(
            location=[lat, lon],
            popup=city,
            fill_color = 'red',
            color = 'clear',
            fill_opacity = 0.5,
            radius=(filtered_data[filtered_data['City'] == city]['Count'].values[0]/2)*10
        ).add_to(india_map)
        
        folium.CircleMarker(
            location=[lat, lon],
            popup=city,
            fill_color = 'black',
            color = 'clear',
            fill_opacity = 1,
            radius=2
        ).add_to(india_map)
    # save the map
    india_map.save('map.html')

    # Create the figure
    fig = px.bar(filtered_data, x='Count', y='City', orientation='h', 
                 title=f'Number of Matches Played in Each City in {selected_season}')
    
    return fig, india_map._repr_html_()


### Rank Layout

In [1518]:
rank_data = pd.read_csv('./Data/Rank_data.csv')
rank_data_fig = px.line(rank_data, x='Season', y='Rank', color='Team', title='Rank of each team over the years')
rank_data_fig.update_layout(yaxis=dict(autorange='reversed'), yaxis2=dict(autorange='reversed', overlaying='y', side='right'))
# options contains team name
options = [{'label': team, 'value': team} for team in rank_data['Team'].unique()]

win_team_each_season = [
    {'Year': '2008', 'Team': 'Rajasthan Royals'},
    {'Year': '2009', 'Team': 'Deccan Chargers'},
    {'Year': '2010', 'Team': 'Chennai Super Kings'},
    {'Year': '2011', 'Team': 'Chennai Super Kings'},
    {'Year': '2012', 'Team': 'Kolkata Knight Riders'},
    {'Year': '2013', 'Team': 'Mumbai Indians'},
    {'Year': '2014', 'Team': 'Kolkata Knight Riders'},
    {'Year': '2015', 'Team': 'Mumbai Indians'},
    {'Year': '2016', 'Team': 'Sunrisers Hyderabad'},
    {'Year': '2017', 'Team': 'Mumbai Indians'},
    {'Year': '2018', 'Team': 'Chennai Super Kings'},
    {'Year': '2019', 'Team': 'Mumbai Indians'},
    {'Year': '2020', 'Team': 'Mumbai Indians'},
    {'Year': '2021', 'Team': 'Chennai Super Kings'},
    {'Year': '2022', 'Team': 'Gujarat Titans'}
]
columns_win_team_each = [{'name': col, 'id': col} for col in win_team_each_season[0].keys()]
display_win_team = html.Div([
    dash_table.DataTable(
        data=win_team_each_season,
        columns=columns_win_team_each,
        style_table={'overflowX': 'auto'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_cell={
            'textAlign': 'left'
        }
    )
])

win_team_each_season_df = pd.DataFrame(win_team_each_season)
win_team_each_season_df = win_team_each_season_df.groupby('Team').count().reset_index()
win_team_each_season_df = win_team_each_season_df.sort_values('Year', ascending=False)
win_team_each_season_df = win_team_each_season_df.rename(columns={'Year': 'Count'})
win_team_each_season_df_fig = px.bar(win_team_each_season_df, x='Count', y='Team', orientation='h', title='Number of times each team won the IPL')
win_team_each_season_df_fig.update_layout(yaxis=dict(autorange='reversed'))


# Create layout
ranking_layout = html.Div(children=[
    html.Div(style=style_title, children=[
        html.H1('Indian Premier League', style=style_IPL),]),
        html.A(html.Button('Go back to home', style=home_button_style), href='/'),
        html.Div(children=[
        html.H1(children='Rank Chart'),
        dcc.Dropdown(
            id='multi-select-dropdown',
            options=options,
            value=[],
            multi=True
        ),
        dcc.Graph(
            id='bump-chart'
            ),
        html.Div(children=[html.H2("Number of season won by each team", style = style_text),
                           dcc.Graph(figure=win_team_each_season_df_fig)], style={'padding':'10px'}),
        html.Div(children=[html.H2("Year-wise winning team", style=style_text),
                           display_win_team], style={'padding':'10px'})
        
    ])
], style=style_main)

@app.callback(
    dash.dependencies.Output('bump-chart', 'figure'),
    dash.dependencies.Input('multi-select-dropdown', 'value'))
def update_bump_chart(selected_values):
    # update the figure based on the selected values
    rank_data_fig = px.line(rank_data[rank_data['Team'].isin(selected_values)], x='Season', y='Rank', color='Team', title='Rank of each team over the years')
    rank_data_fig.update_layout(yaxis=dict(autorange='reversed', tickmode = 'linear', dtick=1), 
                                yaxis2=dict(autorange='reversed', overlaying = 'y', side = 'right', tickmode='linear', dtick=1),
                                xaxis=dict(tickmode='linear', dtick=1))
    return rank_data_fig



### Team Layout

In [1519]:
total_run_per_team = pd.read_csv('./Data/Total_run_per_team_per_Season.csv')

# Create a list of teams for the dropdown
teams = sorted(total_run_per_team['BattingTeam'].unique())

# Define the layout of the app
team_layout = html.Div(children=[
    html.Div(style=style_title, children=[
    html.H1('Indian Premier League', style=style_IPL),]),
    html.A(html.Button('Go back to home', style=home_button_style), href='/'),
        html.Div([
        html.H1("Total Runs Per Season for Team", style = style_text),
        dcc.Dropdown(
            id='team-dropdown-team',
            options=[{'label': team, 'value': team} for team in teams],
            value=teams[0]
        ),
        dcc.Graph(id='line-chart-team')
    ])
], style=style_main)




# Define the callback function to update the line chart based on the selected team
@app.callback(
    Output('line-chart-team', 'figure'),
    [Input('team-dropdown-team', 'value')]
)
def update_line_chart3(selected_team):
    # Filter the dataframe based on the selected team
    filtered_df = total_run_per_team[total_run_per_team['BattingTeam'] == selected_team]
    filtered_df.sort_values('Season', inplace=True)
    # Create a line chart using Plotly Express
    fig = px.line(filtered_df, x='Season', y='total_run', title=f'Total runs per season for {selected_team}')
    return fig

### Summary Layout

In [1520]:
df = pd.read_csv('./Data/match_summary_updated.csv')
summary_layout = html.Div(children=[
    html.Div(style=style_title, children=[
    html.H1('Indian Premier League', style=style_IPL),]),
    html.A(html.Button('Go back to home', style=home_button_style), href='/'),
        html.H1("Match Summary", style=style_text),
        html.Div([
        dcc.Dropdown(
            id='year-dropdown-summary',
            options=[{'label': year, 'value': year} for year in df['Season'].unique()],
            value=df['Season'].unique()[0],
            clearable=False
        ),
        dcc.Dropdown(
            id='team1-dropdown-summary',
            options=[],
            value='',
            clearable=False
        ),
        dcc.Dropdown(
            id='team2-dropdown-summary',
            options=[],
            value='',
            clearable=False
        ),
        dcc.Dropdown(
            id='match-dropdown-summary',
            options=[],
            value='',
            clearable=False
        ),
        dcc.Graph(id='bar-plot-summary-1'),
        html.Div(id='final-score-summary', style={'font-size': '20px', 'font-weight': 'bold', 'margin': '20px'}),
        dcc.Graph(id='bar-plot-summary-2')
        
    ])
], style=style_main)




@app.callback(
    Output('team1-dropdown-summary', 'options'),
    [Input('year-dropdown-summary', 'value')]
)
def update_team1_dropdown_summary(year):
    teams = df[df['Season'] == year]['BattingTeam'].unique()
    options = [{'label': team, 'value': team} for team in teams]
    return options


@app.callback(
    Output('team2-dropdown-summary', 'options'),
    [Input('year-dropdown-summary', 'value'),
    Input('team1-dropdown-summary', 'value')]
)
def update_team2_dropdown_summary(year, team1):
    teams = df[(df['Season'] == year) & (df['BattingTeam'] == team1)]['BowlingTeam'].unique()
    options = [{'label': team, 'value': team} for team in teams]
    return options


@app.callback(
    Output('match-dropdown-summary', 'options'),
    [Input('year-dropdown-summary', 'value'),
    Input('team1-dropdown-summary', 'value'),
    Input('team2-dropdown-summary', 'value')]
)
def update_match_dropdown_summary(year, team1, team2):
    matches = df[(df['Season'] == year) & (df['BattingTeam'] == team1) & (df['BowlingTeam'] == team2)]['ID'].unique()
    options = [{'label': i+1, 'value': match} for i, match in enumerate(matches)]
    return options

# update the bar plot based on the selected match
@app.callback(
    Output('bar-plot-summary-1', 'figure'),
    Output('bar-plot-summary-2', 'figure'),
    Output('final-score-summary', 'children'),
    [Input('year-dropdown-summary', 'value'),
    Input('team1-dropdown-summary', 'value'),
    Input('team2-dropdown-summary', 'value'),
    Input('match-dropdown-summary', 'value')]
)
def update_bar_plot_summary(year, team1, team2, match):
    filtered_df1 = df[(df['Season'] == year) & (df['BattingTeam'] == team1) & (df['BowlingTeam'] == team2) & (df['ID'] == match)]
    filtered_df1.sort_values('overs', inplace=True)
    filtered_df2 = df[(df['Season'] == year) & (df['BattingTeam'] == team2) & (df['BowlingTeam'] == team1) & (df['ID'] == match)]
    filtered_df2.sort_values('overs', inplace=True)
    # plot a grouped bar plot using Plotly Express
    fig1 = go.Figure(data=[
        go.Bar(name=team1, x=filtered_df1['overs'], y=filtered_df1['total_run']),
        go.Bar(name=team2, x=filtered_df2['overs'], y=filtered_df2['total_run'])
    ])
    fig1.update_layout(barmode='group', title=f'Runs scored by {team1} and {team2} in each over on match {match}', 
                      xaxis=dict(title='Overs', tickmode='linear', dtick=1),
                      yaxis=dict(title='Runs'))
    
    
    # plot a grouped line chart with over as x-axis and cumullative runs as y-axis and mark wickets on the line chart
    # find cumulative runs as I don't have that in the dataframe
    filtered_df1['cum_run'] = filtered_df1['total_run'].cumsum()
    filtered_df2['cum_run'] = filtered_df2['total_run'].cumsum()
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=filtered_df1['overs'], 
                              y=filtered_df1['cum_run'], 
                              name=team1, mode = 'lines',
                              line=dict(color='blue')))
    fig2.add_trace(go.Scatter(x=filtered_df1[filtered_df1['isWicketDelivery']!=0]['overs'], 
                              y=filtered_df1[filtered_df1['isWicketDelivery']!=0]['cum_run'], 
                              name=team1, 
                              mode = 'markers',
                              marker=dict(color='blue', size=10)))
    fig2.add_trace(go.Scatter(x=filtered_df2['overs'],
                              y=filtered_df2['cum_run'],
                              name=team2, mode = 'lines',
                              line=dict(color='red')))
    fig2.add_trace(go.Scatter(x=filtered_df2[filtered_df2['isWicketDelivery']!=0]['overs'],
                              y=filtered_df2[filtered_df2['isWicketDelivery']!=0]['cum_run'],
                              name=team2,
                              mode = 'markers',
                              marker=dict(color='red', size=10)))
    # mark wickets on the line chart 
    # fig2.add_trace(go.Scatter(x=filtered_df1[filtered_df1['wicket'] == 1]['overs'], y=filtered_df1[filtered_df1['wicket'] == 1]['cum_run']))
    fig2.update_layout(barmode='group', title=f'Match cummulative run with wicket fall by {team1} and {team2} in match {match}', 
                      xaxis=dict(title='Overs', tickmode='linear', dtick=1),
                      yaxis=dict(title='Runs'))

    total_runs_team1 = filtered_df1['total_run'].sum()
    total_runs_team2 = filtered_df2['total_run'].sum()
    total_wickets_team1 = filtered_df1['isWicketDelivery'].sum()
    total_wickets_team2 = filtered_df2['isWicketDelivery'].sum()
    
    descrition_text = ''
    if total_runs_team1 > total_runs_team2:
        descrition_text = f'{team1} won by'
    elif total_runs_team1 < total_runs_team2:
        descrition_text = f'{team2} won by'
    else:
        descrition_text = f'Match tied'
    score =html.Div(children=[html.Div(f'{team1}: {total_runs_team1}-{total_wickets_team1}'), 
                              html.Div(f'{team2}: {total_runs_team2}-{total_wickets_team2}'), 
                              html.Div(descrition_text)])

    return fig1, fig2, score



### Take URL and return the layout

In [1521]:

@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/batting':
        return batsmen_layout
    elif pathname == '/bowling':
        return bowler_layout
    elif pathname == '/stadium':
        return stadium_layout
    elif pathname == '/rank':
        return ranking_layout
    elif pathname == '/team':
        return team_layout
    elif pathname == '/summary':
        return summary_layout
    else:
        return home_layout

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

if __name__ == '__main__':
    app.run_server(debug=False)